# Imports

In [6]:
import requests
import bs4
import pandas as pd
import re

from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import chromedriver_autoinstaller

from dotenv import dotenv_values

import time

# Install Chromevriver selenium

In [7]:
path = './requirements/chromedriver'
service = Service(executable_path=path)
driver = webdriver.Chrome(service=service)

# Columns and rows display options

In [8]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Configurate dotenv to hide password

In [9]:
config = dotenv_values(".env")

In [10]:
mar_password = config["PASSWORD"]

# Log into Goodreads with Selenium

In [11]:
url = 'https://www.goodreads.com/ap/signin?language=en_US&openid.assoc_handle=amzn_goodreads_web_na&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.mode=checkid_setup&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0&openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.goodreads.com%2Fap-handler%2Fsign-in&siteState=5343dd7ffb96ad0340e3a83b80aba5dc'
driver.get(url)
driver.maximize_window()

In [12]:
username = driver.find_element(By.XPATH, "//input[@type='email']")
username.send_keys("margamontv@gmail.com")

In [13]:
password = driver.find_element(By.XPATH, "//input[@type='password']")
password.send_keys(mar_password)

In [14]:
sign_in = driver.find_element(By.XPATH, "//input[@id='signInSubmit']")
sign_in.click()

# Get the genres urls containing 50 books per page per genre

In [15]:
#Create a genre list with the gernes and a page list with the pages
genre_list = ['fantasy', 'science-fiction']
genre_pages = ['1', '2']

In [16]:
#Create a for loop and dictionaries containing the main urls. This urls contain 50 books each
list_url_genres = []

for g in genre_list:
    for p in genre_pages:
        base_url = 'https://www.goodreads.com/shelf/show/'
        pages_url = f'?page={p}'
        full_url = base_url + g + pages_url
        dict_url = {}
        dict_url['genre'] = g
        dict_url['url'] = full_url
        list_url_genres.append(dict_url)  
list_url_genres

[{'genre': 'fantasy',
  'url': 'https://www.goodreads.com/shelf/show/fantasy?page=1'},
 {'genre': 'fantasy',
  'url': 'https://www.goodreads.com/shelf/show/fantasy?page=2'},
 {'genre': 'science-fiction',
  'url': 'https://www.goodreads.com/shelf/show/science-fiction?page=1'},
 {'genre': 'science-fiction',
  'url': 'https://www.goodreads.com/shelf/show/science-fiction?page=2'}]

In [17]:
# Convert my list of dictionaries (JSON) into a Pandas dataframe
genres_urls = pd.DataFrame(list_url_genres)
genres_urls

,genre,url
0,fantasy,https://www.goodreads.com/shelf/show/fantasy?page=1
1,fantasy,https://www.goodreads.com/shelf/show/fantasy?page=2
2,science-fiction,https://www.goodreads.com/shelf/show/science-fiction?page=1
3,science-fiction,https://www.goodreads.com/shelf/show/science-fiction?page=2


# Obtain the books urls form the genres urls

In [18]:
#Create an empty list for my dictionaries
list_url_books = []

#create a for loop to iterate along the genres urls
for url_i in genres_urls['url']:
    driver_content = driver.get(url_i)
    books_elements = driver.find_elements(By.XPATH, "//a[@class='bookTitle']")
    for element in books_elements:
        dict_url_books = {}
        books_url = element.get_attribute("href")
        dict_url_books['book_url'] = books_url
        list_url_books.append(dict_url_books)

In [19]:
# Convert my list of urls (JSON) into a Pandas dataframe
books_urls = pd.DataFrame(list_url_books)
books_urls

,book_url
0,https://www.goodreads.com/book/show/72193.Harry_Potter_and_the_Philosopher_s_Stone
1,https://www.goodreads.com/book/show/15881.Harry_Potter_and_the_Chamber_of_Secrets
2,https://www.goodreads.com/book/show/5.Harry_Potter_and_the_Prisoner_of_Azkaban
3,https://www.goodreads.com/book/show/5907.The_Hobbit
4,https://www.goodreads.com/book/show/6.Harry_Potter_and_the_Goblet_of_Fire
...,...
195,https://www.goodreads.com/book/show/35519101-rogue-protocol
196,https://www.goodreads.com/book/show/25667918-binti
197,https://www.goodreads.com/book/show/135479.Cat_s_Cradle
198,https://www.goodreads.com/book/show/18966819-golden-son


# Append the genres to my books_urls dataframe throug a for loop

In [20]:
genres_list = []

for book_url_i, row in books_urls.iterrows():
    #print(book_url_i)
    if book_url_i <= 99:
        #print(books_urls.index)
        genres_list.append('Fantasy')
    elif book_url_i >= 100 and book_url_i <= 199:
        genres_list.append('Science Fiction')
print(len(genres_list)) 
#print(genres_list)

200


In [21]:
books_urls['Genre'] = genres_list 
books_urls

,book_url,Genre
0,https://www.goodreads.com/book/show/72193.Harry_Potter_and_the_Philosopher_s_Stone,Fantasy
1,https://www.goodreads.com/book/show/15881.Harry_Potter_and_the_Chamber_of_Secrets,Fantasy
2,https://www.goodreads.com/book/show/5.Harry_Potter_and_the_Prisoner_of_Azkaban,Fantasy
3,https://www.goodreads.com/book/show/5907.The_Hobbit,Fantasy
4,https://www.goodreads.com/book/show/6.Harry_Potter_and_the_Goblet_of_Fire,Fantasy
...,...,...
195,https://www.goodreads.com/book/show/35519101-rogue-protocol,Science Fiction
196,https://www.goodreads.com/book/show/25667918-binti,Science Fiction
197,https://www.goodreads.com/book/show/135479.Cat_s_Cradle,Science Fiction
198,https://www.goodreads.com/book/show/18966819-golden-son,Science Fiction


# Extract the relevant info from my books_urls and append to dictionary

In [22]:
#create an empty list to store my dictionaries
goodreads_books_list = []

#Iterate the urls through a for loop
for books_urls_i in books_urls['book_url']:

#Create a new dictionary to store the info extracted
    goodreads_books = {}
    
#Bring the info from my previous df to my new dictionary
    #goodreads_books['url'] = books_urls['book_url'] 
    #goodreads_books['Genre'] = books_urls['Genre']
#Iterate through the urls
    new_driver_content = driver.get(books_urls_i)

#Extract the relevant info from each book and append to dictionary
    #Title info
    try:
        titles = driver.find_element(By.XPATH, "//h1[@class='Text Text__title1']").text
        goodreads_books['Title'] = titles
    except:
        goodreads_books['Title'] = 'error' 
    #Author info
    try:
        authors = driver.find_element(By.XPATH, "//span[@class='ContributorLink__name']").text
        goodreads_books['Author'] = authors
    except:
        goodreads_books['Author'] = 'error'
    #Rating info
    try:
        ratings = driver.find_element(By.XPATH, "//div[@class='RatingStatistics__rating']").text
        goodreads_books['Rating'] = ratings
    except:
        goodreads_books['Rating'] = 'error' 
    #Ratings count info
    try:
        ratings_count = driver.find_element(By.XPATH, "//span[@data-testid='ratingsCount']").text
        goodreads_books['Ratings count'] = ratings_count
    except: 
        goodreads_books['Ratings count'] = 'error'
    #Reviews count info
    try:
        reviews_count = driver.find_element(By.XPATH, "//span[@data-testid='reviewsCount']").text
        goodreads_books['Reviews count'] = reviews_count
    except: 
        goodreads_books['Reviews count'] = 'error'
    #Pages info
    try:
        pages = driver.find_element(By.XPATH, "//p[@data-testid='pagesFormat']").text
        goodreads_books['Pages'] = pages
    except:
        goodreads_books['Pages'] = 'error'
    #First published info
    try:
        first_published = driver.find_element(By.XPATH, "//p[@data-testid='publicationInfo']").text
        goodreads_books['First published'] = first_published
    except:
        goodreads_books['First published'] = 'error'
    #Cover image info
    try:
        cover_image = driver.find_element(By.XPATH, "//img[@class='ResponsiveImage']")
        image_url = cover_image.get_attribute("src")
        goodreads_books['Cover image'] = image_url
    except:
        goodreads_books['Cover image'] = 'error'   
    time.sleep(1)
    goodreads_books_list.append(goodreads_books)

In [23]:
print(len(goodreads_books_list))

200


In [24]:
# Convert my list of urls (JSON) into a Pandas dataframe
goodreads_books_df = pd.DataFrame(goodreads_books_list)
goodreads_books_df

,Title,Author,Rating,Ratings count,Reviews count,Pages,First published,Cover image
0,Harry Potter and the Philosopher’s Stone,J.K. Rowling,4.47,"9,392,848 ratings","149,406 reviews","223 pages, Paperback","First published June 26, 1997",https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1170803558l/72193.jpg
1,Harry Potter and the Chamber of Secrets,J.K. Rowling,4.43,"3,647,256 ratings","72,580 reviews","341 pages, Hardcover","First published July 2, 1998",https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1474169725i/15881.jpg
2,Harry Potter and the Prisoner of Azkaban,J.K. Rowling,4.58,"3,865,520 ratings","76,425 reviews","435 pages, Mass Market Paperback","First published July 8, 1999",https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1630547330i/5.jpg
3,The Hobbit,J.R.R. Tolkien,4.28,"3,709,210 ratings","64,963 reviews","366 pages, Paperback","First published September 21, 1937",https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1546071216i/5907.jpg
4,Harry Potter and the Goblet of Fire,J.K. Rowling,4.56,"3,442,808 ratings","63,249 reviews","734 pages, Paperback","First published July 8, 2000",https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1554006152i/6.jpg
...,...,...,...,...,...,...,...,...
195,Rogue Protocol,Martha Wells,4.27,"87,849 ratings","7,364 reviews","158 pages, Hardcover","First published August 7, 2018",https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1506001607i/35519101.jpg
196,Binti,Nnedi Okorafor,3.88,"68,409 ratings","9,785 reviews","96 pages, ebook","First published September 22, 2015",https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1433804020i/25667918.jpg
197,Cat's Cradle,Kurt Vonnegut Jr.,4.16,"387,154 ratings","14,071 reviews","179 pages, Paperback","First published January 1, 1963",https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1327867150i/135479.jpg
198,Golden Son,Pierce Brown,4.45,"208,592 ratings","16,720 reviews","466 pages, ebook","First published January 6, 2015",https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1394684475i/18966819.jpg


In [25]:
goodreads_books = pd.concat([goodreads_books_df, books_urls], axis=1)
goodreads_books

,Title,Author,Rating,Ratings count,Reviews count,Pages,First published,Cover image,book_url,Genre
0,Harry Potter and the Philosopher’s Stone,J.K. Rowling,4.47,"9,392,848 ratings","149,406 reviews","223 pages, Paperback","First published June 26, 1997",https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1170803558l/72193.jpg,https://www.goodreads.com/book/show/72193.Harry_Potter_and_the_Philosopher_s_Stone,Fantasy
1,Harry Potter and the Chamber of Secrets,J.K. Rowling,4.43,"3,647,256 ratings","72,580 reviews","341 pages, Hardcover","First published July 2, 1998",https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1474169725i/15881.jpg,https://www.goodreads.com/book/show/15881.Harry_Potter_and_the_Chamber_of_Secrets,Fantasy
2,Harry Potter and the Prisoner of Azkaban,J.K. Rowling,4.58,"3,865,520 ratings","76,425 reviews","435 pages, Mass Market Paperback","First published July 8, 1999",https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1630547330i/5.jpg,https://www.goodreads.com/book/show/5.Harry_Potter_and_the_Prisoner_of_Azkaban,Fantasy
3,The Hobbit,J.R.R. Tolkien,4.28,"3,709,210 ratings","64,963 reviews","366 pages, Paperback","First published September 21, 1937",https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1546071216i/5907.jpg,https://www.goodreads.com/book/show/5907.The_Hobbit,Fantasy
4,Harry Potter and the Goblet of Fire,J.K. Rowling,4.56,"3,442,808 ratings","63,249 reviews","734 pages, Paperback","First published July 8, 2000",https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1554006152i/6.jpg,https://www.goodreads.com/book/show/6.Harry_Potter_and_the_Goblet_of_Fire,Fantasy
...,...,...,...,...,...,...,...,...,...,...
195,Rogue Protocol,Martha Wells,4.27,"87,849 ratings","7,364 reviews","158 pages, Hardcover","First published August 7, 2018",https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1506001607i/35519101.jpg,https://www.goodreads.com/book/show/35519101-rogue-protocol,Science Fiction
196,Binti,Nnedi Okorafor,3.88,"68,409 ratings","9,785 reviews","96 pages, ebook","First published September 22, 2015",https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1433804020i/25667918.jpg,https://www.goodreads.com/book/show/25667918-binti,Science Fiction
197,Cat's Cradle,Kurt Vonnegut Jr.,4.16,"387,154 ratings","14,071 reviews","179 pages, Paperback","First published January 1, 1963",https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1327867150i/135479.jpg,https://www.goodreads.com/book/show/135479.Cat_s_Cradle,Science Fiction
198,Golden Son,Pierce Brown,4.45,"208,592 ratings","16,720 reviews","466 pages, ebook","First published January 6, 2015",https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1394684475i/18966819.jpg,https://www.goodreads.com/book/show/18966819-golden-son,Science Fiction


In [26]:
goodreads_books.to_csv('./data/goodreads_books.csv', index=False)

In [ ]:
driver.get('https://www.goodreads.com/book/show/72193.Harry_Potter_and_the_Philosopher_s_Stone')
author = driver.find_element(By.XPATH, "//span[@class='ContributorLink__name']").text
author

In [ ]:
rating = driver.find_element(By.XPATH, "//div[@class='RatingStatistics__rating']").text
rating

In [ ]:
ratings_count = driver.find_element(By.XPATH, "//span[@data-testid='ratingsCount']").text
ratings_count

In [ ]:
reviews_count = driver.find_element(By.XPATH, "//span[@data-testid='reviewsCount']").text
reviews_count

In [ ]:
pages = driver.find_element(By.XPATH, "//p[@data-testid='pagesFormat']").text
pages

In [ ]:
first_published = driver.find_element(By.XPATH, "//p[@data-testid='publicationInfo']").text
first_published

In [ ]:
image = driver.find_element(By.XPATH, "//img[@class='ResponsiveImage']")
image_url = image.get_attribute("src")
image_url

In [ ]:
# Access each url via BeautifulSoup
for books_urls_i in books_urls['book_url']: 
    #print(books_urls_i)
    response = requests.get(books_urls_i)
    #print(response)
    html = response.content
    parsed_html = bs4.BeautifulSoup(html, "html.parser")
    
# Access Title info and append to dictionary
    title_divs = parsed_html.find_all("h1", {"class": "Text Text__title1"})[0].text
    books_urls['Title'] = title_divs
    
# Access author info and append to dictionary 
    author_divs = parsed_html.find_all("span", {"class": "ContributorLink__name"})[0].text
    books_urls['Author'] = author_divs
    
# Access Rating info and append to dictionary
    rating_divs = parsed_html.find_all("div", {"class": "RatingStatistics__rating"})[0].text
    books_urls['Rating'] = rating_divs 
    
# Access Ratings_Count info and append to dictionary
    ratings_count_divs = parsed_html.find_all("span", {"data-testid": "ratingsCount"})[0].text
    ratings_count_divs_2 = ratings_count_divs.split()[0]
    books_urls['Ratings count'] = ratings_count_divs_2   
    
# Access Reviews_Count info and append to dictionary    
    reviews_count_divs = parsed_html.find_all("span", {"data-testid": "reviewsCount"})[0].text
    reviews_count_divs_2 = rating_count_divs.split()[0]
    books_urls['Reviews count'] = reviews_count_divs_2
    
# Access Nº of pages info and append to dictionary    
    pages_divs = parsed_html.find_all("p", {"data-testid": "pagesFormat"})[0].text
    pages_divs_2 = pages_divs.split()[0]
    books_urls['Pages'] = pages_divs_2
    
# Access First published info and append to dictionary     
    published_divs = parsed_html.find_all("p", {"data-testid": "publicationInfo"})[0].text
    published_divs_2 = published_divs.split()[-1]
    books_urls['First published'] = published_divs_2
    
# Acces Book cover image and append to dictionary
    image_divs = parsed_html.find_all("div", {"class": "ResponsiveImage"})[0].text
    books_urls['Book cover'] = image_divs
books_urls   

In [ ]:
<h1 class="Text Text__title1" data-testid="bookTitle" aria-label="Book title: The Maze Runner">The Maze Runner</h1>

In [ ]:
response = requests.get('https://www.goodreads.com/book/show/72193.Harry_Potter_and_the_Philosopher_s_Stone')
#print(response)
html = response.content
parsed_html = bs4.BeautifulSoup(html, "html.parser")

In [ ]:
#Extract Author particulars
author_divs = parsed_html.find_all("span", {"class": "ContributorLink__name"})[0].text
author_divs

In [ ]:
for books_urls_i in books_urls['book_url']: 
    #print(books_urls_i)
    response = requests.get(books_urls_i)
    #print(response)
    html = response.content
    parsed_html = bs4.BeautifulSoup(html, "html.parser")
#Extract Title particulars
    title_divs = parsed_html.find_all("h1", {"class": "Text Text__title1"})[0].text
title_divs

In [ ]:
#response = requests.get('https://www.goodreads.com/book/show/5.Harry_Potter_and_the_Prisoner_of_Azkaban')
#print(response)
#html = response.content
#parsed_html = bs4.BeautifulSoup(html, "html.parser")
title_divs = parsed_html.find_all("h1", {"class": "Text Text__title1"})[0].text
title_divs

In [ ]:
<h1 class="Text Text__title1" data-testid="bookTitle" aria-label="Book title: Harry Potter and the Prisoner of Azkaban">Harry Potter and the Prisoner of Azkaban</h1>

In [ ]:
<h1 class="Text Text__title1" data-testid="bookTitle" aria-label="Book title: The Maze Runner">The Maze Runner</h1>

In [ ]:
#Extract Rating particulars
rating_divs = parsed_html.find_all("div", {"class": "RatingStatistics__rating"})[0].text
rating_divs

In [ ]:
#Extract Ratings_Count particulars
ratings_count_divs = parsed_html.find_all("span", {"data-testid": "ratingsCount"})[0].text
ratings_count_divs_2 = ratings_count_divs.split()[0]
ratings_count_divs_2

In [ ]:
#Extract Reviews_Count particulars
reviews_count_divs = parsed_html.find_all("span", {"data-testid": "reviewsCount"})[0].text
reviews_count_divs_2 = rating_count_divs.split()[0]
reviews_count_divs_2

In [ ]:
#Extract Pages particulars
pages_divs = parsed_html.find_all("p", {"data-testid": "pagesFormat"})[0].text
pages_divs_2 = pages_divs.split()[0]
pages_divs_2 

In [ ]:
#Extract First Published particulars
published_divs = parsed_html.find_all("p", {"data-testid": "publicationInfo"})[0].text
published_divs_2 = published_divs.split()[-1]
published_divs_2 

In [ ]:
for url_i in genres_urls['url']:
    for genre_i in genres_urls['genre']:
        h1_element = driver.find_element(By.TAG_NAME, "h1")
        # Get the text from the <h1> element
        text = h1_element.text
        text
        
    

In [ ]:
list_genres_books = []


#create a for loop to iterate along the genres urls
for genre_i in genres_urls['genre']:
    driver_content = driver.get(url_i)
    books_elements = driver.find_elements(By.XPATH, "//a[@class='bookTitle']")
    for element in books_elements:
        dict_url_books = {}
        books_url = element.get_attribute("href")
        dict_url_books['book_url'] = books_url
        list_url_books.append(dict_url_books)

list_genres_books

In [ ]:
#Create an empty list for my dictionaries
list_url_books = []


#create a for loop to iterate along the genres urls
for url_i in genres_urls['url']:
    for genre_g in genres_urls['genre']:
        driver_content = driver.get(url_i)
        books_elements = driver.find_elements(By.XPATH, "//a[@class='bookTitle']")
        for element in books_elements:
            dict_url_books = {}
            dict_url_books['genre'] = genre_g
            books_url = element.get_attribute("href")
            dict_url_books['book_url'] = books_url
            list_url_books.append(dict_url_books)
list_url_books

   
    
    #
    
    #print(f'These are books urls {dict_url_books["book_url"]}')
    #for g in genres_urls["genre"]:
        #dict_url_books['genre'] = g
        #print(f'These are book genres {dict_url_books["genre"]}')
    
    #list_url_books.append(dict_url_books) 
#list_url_books   


    
    #Get the response and parsed_html from the genres urls 
    #browser = webdriver.Chrome()
    #browser.get(url_i)
    #html = browser.page_source
    #print(html)
    #browser.close()
    #page_source = driver.get(url_i)
    #soup = bs4.BeautifulSoup(page_source, 'html.parser')
    #response = driver.get(url_i)
    #print(response)
    #html = response.content
    #print(html)
    #parsed_html = bs4.BeautifulSoup(html, "html.parser")
    
  #Access the individual url of every book
    # First layer: The element that contains all the data
    #divs = parsed_html.find_all("div", {"class": "elementList"})
    # Second layer: Extracting html tags that contain the links
    #links = [div.find('a') for div in divs]
    # Extracting the partial links  
    #relative_url = [link['href'] for link in links]  
    #print(relative_url)
    # Computing the full url addresses 
    #full_url_books = [urljoin(url_i, relativeurl) for relativeurl in relative_url]
    #print(full_url_books)
    
    # Filter only the book links
    #book_url = [i for i in full_url_books if "https://www.goodreads.com/book/show" in i]  

In [ ]:
browser = webdriver.Chrome()
browser.get('https://www.goodreads.com/shelf/show/fantasy?page=7')
html = browser.page_source
print(html)
browser.close()

In [ ]:
response = requests.get('https://www.goodreads.com/shelf/show/fantasy', params={'page': 2}, allow_redirects = False)
print(response.url)
html = response.content
print(html)
parsed_html = bs4.BeautifulSoup(html, "html.parser")

In [ ]:
response.headers

In [ ]:
response.request.headers

In [ ]:
curl 'https://www.goodreads.com/shelf/show/fantasy?page=2' \
  -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7' \
  -H 'Accept-Language: es-ES,es;q=0.9,en;q=0.8,ca;q=0.7' \
  -H 'Connection: keep-alive' \
  -H 'Cookie: ccsid=833-3280815-4595007; p=A2JpLvFUzHoFnkGDktpVsrneQvCz1yyfn9cHjrK7b6kvrNHf; session-id=137-2093370-0067327; ubid-main=131-8734629-7675128; logged_out_browsing_page_count=2; session-id-time=2287772909l; session-token=TDkgKJjK0q6Qy7z7iSAIWonvvdBjuNSU8vxXGqhsyxdRd4vk7+rlDjnSQQQ4BJvBKzRhSFCmIgKBn5DZWfro/7Sv2UuPDuMwr6pnoT0urYu41KbKooa8V6fii3cYGfhvcZSiTh3Djy3CBHXKihqET1KIJI4fOhuiWXaH7pNx3FJiG2MLV9ZXnbla4oPMYwSUkbSO5EVxFM/YIc6v+B+ru/6DK5f5Slwee0ER1pxtYPF5z0ykD0lc6ehj0VH7NVTr; x-main="LL8mIGhhNokiP8VAe0bWw4thkO?8vcfj2XrZ3rbT@cfWmkhtJ9GR99MiuACYTDau"; at-main=Atza|IwEBIJCtMSjIcifxjtXd9FM9WPeK6FS3ZCsdOXMxps2oo1pMtedsj4pScJJ9b7b-PHosYC8tGjVW9UO-pVLH82BMiTXgjjBipjjgu3tHabxqQmS1N_yuGF98iYvn-B5evKUP_AE9G2CQ33zRfshct3lqHcTtLQ8VEvl0ll4dytVzYT-MBsjuISZG6oT_fqRm6O4eTWFBOWqbQOI4cDMEQindJTbsDyYva6m05r3glNsaRoEIIVxLxc6Ci1i16Hlav2C1L7Y; sess-at-main="mxOxHSYj6mAlw4kJMZorUzqxRxdjotqsq+Hr6gJJPac="; __qca=P0-784740780-1666448796004; srb_8=0_ar; csm-sid=336-5163223-8746583; locale=es; _session_id2=510eb2330b9531818f2929bbe3f095ec; __gads=ID=601ea9b2c2470b3a:T=1675587371:RT=1688405261:S=ALNI_MayFXVo0aiMMK2l8sRautsDWUAT9Q; __gpi=UID=00000bd4200f6260:T=1675587371:RT=1688405261:S=ALNI_MYCAtihy5YRMxHzO_fi7kEo22eZqg' \
  -H 'If-None-Match: W/"16277fe7a97b584eb683dc13ea73552c"' \
  -H 'Referer: https://www.goodreads.com/shelf/show/fantasy' \
  -H 'Sec-Fetch-Dest: document' \
  -H 'Sec-Fetch-Mode: navigate' \
  -H 'Sec-Fetch-Site: same-origin' \
  -H 'Sec-Fetch-User: ?1' \
  -H 'Upgrade-Insecure-Requests: 1' \
  -H 'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36' \
  -H 'sec-ch-ua: "Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"' \
  -H 'sec-ch-ua-mobile: ?0' \
  -H 'sec-ch-ua-platform: "Windows"' \
  --compressed

In [ ]:
GET /shelf/show/fantasy?page=3 HTTP/1.1
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7
Accept-Encoding: gzip, deflate, br
Accept-Language: es-ES,es;q=0.9,en;q=0.8,ca;q=0.7
Connection: keep-alive
Cookie: ccsid=833-3280815-4595007; p=A2JpLvFUzHoFnkGDktpVsrneQvCz1yyfn9cHjrK7b6kvrNHf; session-id=137-2093370-0067327; ubid-main=131-8734629-7675128; logged_out_browsing_page_count=2; session-id-time=2287772909l; session-token=TDkgKJjK0q6Qy7z7iSAIWonvvdBjuNSU8vxXGqhsyxdRd4vk7+rlDjnSQQQ4BJvBKzRhSFCmIgKBn5DZWfro/7Sv2UuPDuMwr6pnoT0urYu41KbKooa8V6fii3cYGfhvcZSiTh3Djy3CBHXKihqET1KIJI4fOhuiWXaH7pNx3FJiG2MLV9ZXnbla4oPMYwSUkbSO5EVxFM/YIc6v+B+ru/6DK5f5Slwee0ER1pxtYPF5z0ykD0lc6ehj0VH7NVTr; x-main="LL8mIGhhNokiP8VAe0bWw4thkO?8vcfj2XrZ3rbT@cfWmkhtJ9GR99MiuACYTDau"; at-main=Atza|IwEBIJCtMSjIcifxjtXd9FM9WPeK6FS3ZCsdOXMxps2oo1pMtedsj4pScJJ9b7b-PHosYC8tGjVW9UO-pVLH82BMiTXgjjBipjjgu3tHabxqQmS1N_yuGF98iYvn-B5evKUP_AE9G2CQ33zRfshct3lqHcTtLQ8VEvl0ll4dytVzYT-MBsjuISZG6oT_fqRm6O4eTWFBOWqbQOI4cDMEQindJTbsDyYva6m05r3glNsaRoEIIVxLxc6Ci1i16Hlav2C1L7Y; sess-at-main="mxOxHSYj6mAlw4kJMZorUzqxRxdjotqsq+Hr6gJJPac="; __qca=P0-784740780-1666448796004; srb_8=0_ar; csm-sid=336-5163223-8746583; locale=es; _session_id2=510eb2330b9531818f2929bbe3f095ec; __gads=ID=601ea9b2c2470b3a:T=1675587371:RT=1688405261:S=ALNI_MayFXVo0aiMMK2l8sRautsDWUAT9Q; __gpi=UID=00000bd4200f6260:T=1675587371:RT=1688405261:S=ALNI_MYCAtihy5YRMxHzO_fi7kEo22eZqg
Host: www.goodreads.com
Referer: https://www.goodreads.com/shelf/show/fantasy?page=2
Sec-Fetch-Dest: document
Sec-Fetch-Mode: navigate
Sec-Fetch-Site: same-origin
Sec-Fetch-User: ?1
Upgrade-Insecure-Requests: 1
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36
sec-ch-ua: "Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"
sec-ch-ua-mobile: ?0
sec-ch-ua-platform: "Windows"


In [ ]:
$session = New-Object Microsoft.PowerShell.Commands.WebRequestSession
$session.UserAgent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
$session.Cookies.Add((New-Object System.Net.Cookie("ccsid", "833-3280815-4595007", "/", "www.goodreads.com")))
$session.Cookies.Add((New-Object System.Net.Cookie("p", "A2JpLvFUzHoFnkGDktpVsrneQvCz1yyfn9cHjrK7b6kvrNHf", "/", "www.goodreads.com")))
$session.Cookies.Add((New-Object System.Net.Cookie("session-id", "137-2093370-0067327", "/", ".goodreads.com")))
$session.Cookies.Add((New-Object System.Net.Cookie("ubid-main", "131-8734629-7675128", "/", ".goodreads.com")))
$session.Cookies.Add((New-Object System.Net.Cookie("logged_out_browsing_page_count", "2", "/", "www.goodreads.com")))
$session.Cookies.Add((New-Object System.Net.Cookie("session-id-time", "2287772909l", "/", ".goodreads.com")))
$session.Cookies.Add((New-Object System.Net.Cookie("session-token", "TDkgKJjK0q6Qy7z7iSAIWonvvdBjuNSU8vxXGqhsyxdRd4vk7+rlDjnSQQQ4BJvBKzRhSFCmIgKBn5DZWfro/7Sv2UuPDuMwr6pnoT0urYu41KbKooa8V6fii3cYGfhvcZSiTh3Djy3CBHXKihqET1KIJI4fOhuiWXaH7pNx3FJiG2MLV9ZXnbla4oPMYwSUkbSO5EVxFM/YIc6v+B+ru/6DK5f5Slwee0ER1pxtYPF5z0ykD0lc6ehj0VH7NVTr", "/", ".goodreads.com")))
$session.Cookies.Add((New-Object System.Net.Cookie("x-main", "`"LL8mIGhhNokiP8VAe0bWw4thkO?8vcfj2XrZ3rbT@cfWmkhtJ9GR99MiuACYTDau`"", "/", ".goodreads.com")))
$session.Cookies.Add((New-Object System.Net.Cookie("at-main", "Atza|IwEBIJCtMSjIcifxjtXd9FM9WPeK6FS3ZCsdOXMxps2oo1pMtedsj4pScJJ9b7b-PHosYC8tGjVW9UO-pVLH82BMiTXgjjBipjjgu3tHabxqQmS1N_yuGF98iYvn-B5evKUP_AE9G2CQ33zRfshct3lqHcTtLQ8VEvl0ll4dytVzYT-MBsjuISZG6oT_fqRm6O4eTWFBOWqbQOI4cDMEQindJTbsDyYva6m05r3glNsaRoEIIVxLxc6Ci1i16Hlav2C1L7Y", "/", ".goodreads.com")))
$session.Cookies.Add((New-Object System.Net.Cookie("sess-at-main", "`"mxOxHSYj6mAlw4kJMZorUzqxRxdjotqsq+Hr6gJJPac=`"", "/", ".goodreads.com")))
$session.Cookies.Add((New-Object System.Net.Cookie("__qca", "P0-784740780-1666448796004", "/", ".goodreads.com")))
$session.Cookies.Add((New-Object System.Net.Cookie("srb_8", "0_ar", "/", "www.goodreads.com")))
$session.Cookies.Add((New-Object System.Net.Cookie("csm-sid", "336-5163223-8746583", "/", "www.goodreads.com")))
$session.Cookies.Add((New-Object System.Net.Cookie("locale", "es", "/", "www.goodreads.com")))
$session.Cookies.Add((New-Object System.Net.Cookie("_session_id2", "510eb2330b9531818f2929bbe3f095ec", "/", "www.goodreads.com")))
$session.Cookies.Add((New-Object System.Net.Cookie("__gads", "ID=601ea9b2c2470b3a:T=1675587371:RT=1688405261:S=ALNI_MayFXVo0aiMMK2l8sRautsDWUAT9Q", "/", ".goodreads.com")))
$session.Cookies.Add((New-Object System.Net.Cookie("__gpi", "UID=00000bd4200f6260:T=1675587371:RT=1688405261:S=ALNI_MYCAtihy5YRMxHzO_fi7kEo22eZqg", "/", ".goodreads.com")))
Invoke-WebRequest -UseBasicParsing -Uri "https://www.goodreads.com/shelf/show/fantasy?page=3" `
-WebSession $session `
-Headers @{
"Accept"="text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7"
  "Accept-Encoding"="gzip, deflate, br"
  "Accept-Language"="es-ES,es;q=0.9,en;q=0.8,ca;q=0.7"
  "Referer"="https://www.goodreads.com/shelf/show/fantasy?page=2"
  "Sec-Fetch-Dest"="document"
  "Sec-Fetch-Mode"="navigate"
  "Sec-Fetch-Site"="same-origin"
  "Sec-Fetch-User"="?1"
  "Upgrade-Insecure-Requests"="1"
  "sec-ch-ua"="`"Not.A/Brand`";v=`"8`", `"Chromium`";v=`"114`", `"Google Chrome`";v=`"114`""
  "sec-ch-ua-mobile"="?0"
  "sec-ch-ua-platform"="`"Windows`""
}

In [ ]:
#Create an empty list for my dictionaries
list_url_books = []

#create a for loop to iterate along the genres urls
for i in genres_urls['url']:
    print("----URL---->", i)
    
    #Get the response and parsed_html from the genres urls 
    response = requests.get(i)
    print(response)
    html = response.content
    print
    parsed_html = bs4.BeautifulSoup(html, "html.parser")
    
    #Access the individual url of every book
    # First layer: The element that contains all the data
    divs = parsed_html.find_all("div", {"class": "elementList"})
    # Second layer: Extracting html tags that contain the links
    links = [div.find('a') for div in divs]
    # Extracting the partial links  
    relative_url = [link['href'] for link in links] 
    print(relative_url)
    # Computing the full url addresses 
    full_url_books = [urljoin(i, relativeurl) for relativeurl in relative_url]
    # Filter only the book links
    book_url = [i for i in full_url_books if "https://www.goodreads.com/book/show" in i]
    #print(full_url_books)
    
"""   
    #Create empty dictionary and columns for the genres and books urls
    dict_url_books = {}
    dict_url_books['book_url'] = book_url
    #print(f'These are books urls {dict_url_books["book_url"]}')
    for g in genres_urls["genre"]:
        dict_url_books['genre'] = g
        print(f'These are book genres {dict_url_books["genre"]}')
    
    list_url_books.append(dict_url_books) 
"""   

In [ ]:
list_url_books

In [ ]:
base_url = 'https://www.goodreads.com/shelf/show/'
genre = 'fantasy'
sample_page = '1'
pages = f'?page={sample_page}'

In [ ]:
#url_fantasy = 'https://www.goodreads.com/shelf/show/fantasy'
response = requests.get(base_url + genre + pages)
html = response.content
parsed_html = bs4.BeautifulSoup(html, "html.parser")

In [ ]:
book_fantasy_url = []

for i in range(25):
    book_fantasy_url.append(f'https://www.goodreads.com/shelf/show/fantasy?page={i+1}')
book_fantasy_url    

In [ ]:
sample_page = '1'

list_url = []
dict_url = {}
base_url = 'https://www.goodreads.com/shelf/show/'
genre = 'fantasy'
pages = f'?page={sample_page}'
full_url = base_url + genre + pages

dict_url['genre'] = genre
dict_url['url'] = full_url

list_url.append(dict_url)

list_url

In [ ]:
book_title = []
book_author = []
book_rating = []
book_ratings_count = []
book_reviews_count = []
book_genres = []
book_pages = []
book_first_published = []

for i in book_fantasy_url:
    author_divs = parsed_html.find_all("span", {"class": "ContributorLink__name"})[0].text
    book_title.append(title_text)
    
    

In [ ]:
book_list = []

book_info = {}
book_info['genre'] = genre
book_info['author'] = parsed_html.find_all("span", {"class": "ContributorLink__name"})[0].text
book_info['title'] = parsed_html.find_all("h1", {"class": "Text Text__title1"})[0].text

book_list.append(book_info)
book_list

In [ ]:

url = 'https://www.goodreads.com/shelf/show/fantasy'

"""

"""

In [ ]:
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0'}

In [ ]:
response = requests.get(url,
                        headers=headers)
response

In [ ]:
html = response.content
parsed_html = bs4.BeautifulSoup(html, "html.parser")
#parsed_html

In [ ]:
parsed_tags = list(set([tag.name for tag in parsed_html.find_all(True)]))
print(type(parsed_tags), '\n')
print(parsed_tags)

In [ ]:
book_title = []
book_author = []
book_rating = []
book_ratings_count = []
book_reviews_count = []
book_genres = []
book_pages = []
book_first_published = []

In [ ]:
#Extract Author particulars
    author_divs = note_soup.find_all("a", 
    {"Class":"ContributorLink__name"})
    print(author_divs)

In [ ]:
#creating a loop counter
i = 0

#Loop through all 50 books
for url in book_url:
    
    #connect to url page
    note_resp = requests.get(url)
    
    #checking if the request is successful
    if note_resp.status_code == 200:
        print("URL{}: {}".format(i+1, url))
    
    else:
        print('Status code{}: Skipping URL #{}: {}'.format(note_resp.status_code, i+1, url))
        i = i+1
        continue
    
    #get HTML from url page
    note_html = note_resp.content
    
    #create beautifulsoup object for url page
    note_soup = bs4.BeautifulSoup(note_html, 'html.parser')
    
    #Extract Author particulars
    author_divs = note_soup.find_all("a", 
    {"Class":"ContributorLink__name"})
    print(author_divs)
    

In [ ]:
author_text = author_divs[0].find_all('a')[0].text
    book_author.append(author_text)
    
    #Extract title particulars
    title_divs = note_soup.find_all("div", {"class": "last col"})
    title_text = title_divs[0].find_all('h1')[0].text
    book_title.append(title_text)
    
    #Extract rating particulars
    rating_divs = note_soup.find_all("div", {"class": "uitext stacked", "id": "bookMeta"})
    rating_text = rating_divs[0].find_all("span", {"itemprop": "ratingValue"})[0].text
    book_rating.append(rating_text)
    
    #Extracting page particulars
    page_divs = note_soup.find_all("div", {"class": "row"})
    try:
        page_text = page_divs[0].find_all("span", {"itemprop": "numberOfPages"})[0].text.strip(' pages')
    except IndexError:
        page_text = 0
    book_pages.append(page_text)
    
    #Incremeting the loop counter
    i = i+1

In [ ]:
"""
LOGIN_URL = "https://www.goodreads.com/user/sign_in"



def get_authenticity_token(html):
    soup = bs4.BeautifulSoup(html, "html.parser")
    token = soup.find('input', attrs={'name': 'authenticity_token'})
    if not token:
        print('could not find `authenticity_token` on login form')
    return token.get('value').strip()



def get_login_n(html):
    soup = bs4.BeautifulSoup(html, "html.parser")
    n = soup.find('input', attrs={'name': 'n'})
    if not n:
        print('could not find `n` on login form')
    return n.get('value').strip()


email = ""  # login email
password = ""  # login password

payload = {
    'user[email]': email,
    'user[password]': password,
    'utf8': '&#x2713;',
}

session = requests.Session()
session.headers = {'User-Agent': ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) '
    'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36')}
response = session.get(LOGIN_URL)


token = get_authenticity_token(response.text)
n = get_login_n(response.text)
payload.update({
    'authenticity_token': token,
    'n': n
})


print(f"attempting to log in as {email}")
p = session.post(LOGIN_URL, data=payload)  # perform login
"""

In [ ]:
"""
logged_in_page = session.get('https://www.goodreads.com/shelf/show/fantasy?page=2')
html = logged_in_page.content
html
"""

In [ ]:
"""
payload = {
    'email': '',
    'password': ''
}
"""

In [ ]:
#login_request = session.post('https://www.goodreads.com/ap/signin?openid.return_to=https%3A%2F%2Fwww.goodreads.com%2Fap-handler%2Fregister&prevRID=3FKV41QR960E7V1VC7Z8&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.assoc_handle=amzn_goodreads_web_na&openid.mode=checkid_setup&siteState=2b6b0add19b28ac29e1debe93b340eba&language=en_US&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&pageId=amzn_goodreads_web_na&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0', data=payload)

In [ ]:
"""
if login_request.status_code == 200:
    # Login successful
    logged_in_page = session.get('https://www.goodreads.com/shelf/show/fantasy?page=2')
    # Continue with scraping the desired page
    # ...
else:
    # Login failed
    print('Login unsuccessful')
"""

In [ ]:
"""
html = logged_in_page.content
html
#parsed_html = bs4.BeautifulSoup(html, "html.parser")
#parsed_html
"""